In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [22]:
metal_files_list_f <- 'metal2plink.lst'
phe_info_f <- '../../05_gbe//phenotype_info.tsv'
out_f <- 'icdinfo.metal.20200717.txt'


In [7]:
metal_files_list_f %>%
fread(head=F) %>% 
mutate(V1 = str_replace(basename(V1), '.metal.tsv.gz', '')) %>% 
pull() -> metal_GBE_IDs


In [9]:
phe_info_f %>% fread() %>% 
rename('GBE_ID' = '#GBE_ID') -> phe_info_df


In [14]:
phe_info_df %>% 
select(
    GBE_ID, N_GBE, N_NBW, N_AFR, N_EAS, N_SAS, N_SMR, N_OTH
) %>%
gather(pop_str, N, -GBE_ID) %>%
group_by(GBE_ID) %>%
summarise(N_MA = sum(N)) %>%
ungroup() -> N_MA_df 


`summarise()` ungrouping output (override with `.groups` argument)



In [19]:
phe_info_df %>% 
filter(GBE_ID %in% metal_GBE_IDs) %>%
select(GBE_ID, GBE_NAME) %>% 
left_join(N_MA_df) %>%
rename('N' = 'N_MA') -> icdinfo_df

Joining, by = "GBE_ID"



In [23]:
icdinfo_df %>%
mutate(N2=N, N3=N, Y='Y') %>%
select(GBE_ID, N, GBE_NAME, N2, N3, Y) %>%
fwrite(out_f, sep='\t', na = "NA", quote=F, col.names=F)
